<a href="https://colab.research.google.com/github/BuyukCem/-Deep-Learning-chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
data = {
    "intents": [
                {
                    "tag":"name",
                    "patterns": ["Quel est ton prénom?","Comment tu t'appelles", "Qui es-tu?"],
                    "responses": ["Mon prénom est Alpha", "Je suis Alpha", "Alpha"]
                },
              {"tag": "greeting",
                "patterns": ["Hello", "La forme?", "yo", "Salut", "ça roule?"],
                "responses": ["Salut à toi!", "Hello", "Comment vas tu?", "Salutations!", "Enchanté"],
             },
             {"tag": "age",
              "patterns": ["Quel âge as-tu?", "C'est quand ton anniversaire?", "Quand es-tu né?"],
              "responses": ["J'ai 25 ans", "Je suis né en 1996", "Ma date d'anniversaire est le 3 juillet et je suis né en 1996", "03/07/1996"]
             },
             {"tag": "date",
              "patterns": ["Que fais-tu ce week-end?",
"Tu veux qu'on fasse un truc ensemble?", "Quels sont tes plans pour cette semaine"],
              "responses": ["Je suis libre toute la semaine", "Je n'ai rien de prévu", "Je ne suis pas occupé"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "Salut", "see ya", "adios", "cya"],
              "responses": ["C'était sympa de te parler", "à plus tard", "On se reparle très vite!"]
             }
    ]
}

In [5]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
words = []
classes = []
doc_y = []
doc_x = []


for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    doc_x.append(pattern)
    doc_y.append(intent["tag"])

  if intent["tag"] not in classes:
    classes.append(intent["tag"])


words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

words = sorted(set(words))
classes = sorted(set(classes))


In [11]:
print(words)
print(classes)
print(doc_y)
print(doc_x)

['adios', 'anniversaire', 'as-tu', 'bye', "c'est", 'ce', 'cette', 'comment', 'cya', 'ensemble', 'es-tu', 'est', 'fais-tu', 'fasse', 'forme', 'hello', 'la', 'né', 'plan', 'pour', 'prénom', "qu'on", 'quand', 'que', 'quel', 'quels', 'qui', 'roule', 'salut', 'see', 'semaine', 'sont', "t'appelles", 'te', 'ton', 'truc', 'tu', 'un', 'veux', 'week-end', 'ya', 'yo', 'âge', 'ça']
['age', 'date', 'goodbye', 'greeting', 'name']
['name', 'name', 'name', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']
['Quel est ton prénom?', "Comment tu t'appelles", 'Qui es-tu?', 'Hello', 'La forme?', 'yo', 'Salut', 'ça roule?', 'Quel âge as-tu?', "C'est quand ton anniversaire?", 'Quand es-tu né?', 'Que fais-tu ce week-end?', "Tu veux qu'on fasse un truc ensemble?", 'Quels sont tes plans pour cette semaine', 'bye', 'Salut', 'see ya', 'adios', 'cya']


In [19]:
training = []
out_empty = [0] * len(classes)

for idx, doc in enumerate(doc_x):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
        
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1

    training.append([bow, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [22]:
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])

epochs = 200


model = Sequential()

model.add(Dense(128,input_shape=input_shape, activation="relu"))

model.add(Dropout(0.5))
model.add(Dense(output_shape, activation="softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [23]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               5760      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                                 
Total params: 6,405
Trainable params: 6,405
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 818ms/step - loss: 1.6071 - accuracy: 0.2105
Epoch 2/200
1/1 [==============================] - 0s 10ms/step - loss: 1.5533 - accuracy: 0.3684
Epoch 3/200
1/1 [==============================] - 0s 10ms/step - loss: 1.4006 - accuracy: 0.5263
Epoch 4/200
1/1 [==============================] - 0s 10ms/step - loss: 1.3507 - accuracy: 0.5789
Epoch 5/200
1/1 [==============================] - 0s 11ms/step - loss: 1.2173 - accuracy: 0.6842
Epoch 6/200
1/1 [==============================] - 0s 11ms/step - loss: 1.0794 - accuracy: 0.9474
Epoch 7/200
1/1 [==============================] - 0s 11ms/step - loss: 0.9745 - accuracy: 0.8947
Epoch 8/200
1/1 [==============================] - 0s 9ms/step - loss: 0.9320 - accuracy: 0.9474
Epoch 9/200
1/1 [==============================] - 0s 9ms/step - loss: 0.7519 - accuracy: 0.9474
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 0.7350 - accuracy: 0.9474
Epoch 11/200
1/1 [===

In [25]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result
  

In [ ]:


while True:
    message = input("")
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

Salut
C'était sympa de te parler
Quel est ton prénom?
Je suis Alpha
Quel âge as-tu?
Ma date d'anniversaire est le 3 juillet et je suis né en 1996
Que fais-tu ce weekend?
Je ne suis pas occupé
Bye
à plus tard
